In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = ''
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    devs = heats_data.loc['deviation'].to_numpy()
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    else:
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    #import scipy.stats

    dif_conc = [] 
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) + k])
               
    # transforming deviation to the matrix form
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs))) 

    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)

    # finding vector of experimental values

    q_corr = heats
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            q_corr[i] -= volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j]

    A = (volumes[dH_ind_wtf] * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) 
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    # calculating st. dev. of dH_res / checking the quality of linear regression

    F_min = 0
    for i in range(len(heats)):
        F_min += (np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
    #print(sum(heats))
    print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    #disp_0 = F_min / (len(heats) - len(dH_ind_wtf))

    #tn = scipy.stats.t.ppf((1 + 0.95) / 2, len(heats) - len(dH_ind_wtf))

    #disp = []

    #for i in range(len(dH_ind_wtf)):
        #disp.append(disp_0 * LTP[i, i])
    
    return F_min, dH_res_out #disp

In [60]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_1.xlsx"
_calor_setup_type = 'DSC' # use 'DSC' for test_1 and test_2; use 'ampoule' for test_3

max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.0001, 0.5

# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)

#----
lg_k[tar_num] = 7 # it is just for quick variation of initial approximation of the constant to be optimized. Delete in the end
#----
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

F_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff)

new_min = F_min

[[7.97022000e-04 1.00000000e-18]
 [7.95695999e-04 9.91228990e-05]
 [7.94374403e-04 1.97916525e-04]
 [7.93057190e-04 2.96382517e-04]
 [7.91744338e-04 3.94522502e-04]
 [7.90435825e-04 4.92338095e-04]
 [7.89131631e-04 5.89830903e-04]
 [7.87831733e-04 6.87002520e-04]
 [7.86536110e-04 7.83854532e-04]
 [7.85244742e-04 8.80388513e-04]
 [7.83957608e-04 9.76606026e-04]
 [7.82674686e-04 1.07250863e-03]
 [7.81395956e-04 1.16809785e-03]
 [7.80121398e-04 1.26337525e-03]
 [7.78850991e-04 1.35834233e-03]
 [7.77584715e-04 1.45300060e-03]]
[0.0 0.0 43.820814384706274]


In [43]:
print(c_res_out)
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[array([7.97022000e-04, 1.11479986e-19, 8.88520015e-19]), array([7.08828224e-04, 1.22551237e-05, 8.68677754e-05]), array([6.23801896e-04, 2.73440187e-05, 1.70572507e-04]), array([5.42783875e-04, 4.61092023e-05, 2.50273315e-04]), array([4.66824115e-04, 6.96022790e-05, 3.24920223e-04]), array([3.97133182e-04, 9.90354522e-05, 3.93302643e-04]), array([0.00033492, 0.00013562, 0.00045421]), array([0.0002811 , 0.00018027, 0.00050673]), array([0.00023598, 0.0002333 , 0.00055055]), array([0.00019915, 0.0002943 , 0.00058609]), array([0.0001696 , 0.00036225, 0.00061436]), array([0.00014605, 0.00043589, 0.00063662]), array([0.00012727, 0.00051397, 0.00065413]), array([0.00011218, 0.00059543, 0.00066794]), array([9.99271856e-05, 6.79418520e-04, 6.78923805e-04]), array([8.98650046e-05, 7.65280894e-04, 6.87719710e-04])]


In [51]:
print('\nStoich coeff data')
print(st_coeff_data)

print('\nlg K')
print(lg_k_data)

print('\nconcentrations')
print(con_data)

print('\ntype con')
print(type_con)

print('\nexperimental data')
print(heats_data)

print('\ntargets')
print(targets)

print('\ndelta_H')
print(delta_H)


Stoich coeff data
   PLP  T3H  name
0    1    1  Comp

lg K
   lg_k
0  5.13

concentrations
         PLP           T3H series
0   0.000797  1.000000e-18       
1   0.000796  9.912290e-05       
2   0.000794  1.979165e-04       
3   0.000793  2.963825e-04       
4   0.000792  3.945225e-04       
5   0.000790  4.923381e-04       
6   0.000789  5.898309e-04       
7   0.000788  6.870025e-04       
8   0.000787  7.838545e-04       
9   0.000785  8.803885e-04       
10  0.000784  9.766060e-04       
11  0.000783  1.072509e-03       
12  0.000781  1.168098e-03       
13  0.000780  1.263375e-03       
14  0.000779  1.358342e-03       
15  0.000778  1.453001e-03       

type con
0    tot
1    tot
Name: 0, dtype: object

experimental data
                     1           2           3            4           5   \
data                                                                       
volumes          15.025       15.05      15.075         15.1      15.125   
observation    0.069951    0.06

In [52]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[16]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]]

prod names
['PLP', 'T3H', 'Comp']
['PLP', 'T3H']

const
[[0.]
 [0.]
 [4.]]

concentrations
[[7.97022000e-04 1.00000000e-18]
 [7.95695999e-04 9.91228990e-05]
 [7.94374403e-04 1.97916525e-04]
 [7.93057190e-04 2.96382517e-04]
 [7.91744338e-04 3.94522502e-04]
 [7.90435825e-04 4.92338095e-04]
 [7.89131631e-04 5.89830903e-04]
 [7.87831733e-04 6.87002520e-04]
 [7.86536110e-04 7.83854532e-04]
 [7.85244742e-04 8.80388513e-04]
 [7.83957608e-04 9.76606026e-04]
 [7.82674686e-04 1.07250863e-03]
 [7.81395956e-04 1.16809785e-03]
 [7.80121398e-04 1.26337525e-03]
 [7.78850991e-04 1.35834233e-03]
 [7.77584715e-04 1.45300060e-03]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object

volumes
[15.024997 15.049994 15.074991 15.099988 15.124985 15.149982 15.174979
 15.199976 15.224973 15.24997 15.274967 15.299964 15.324961 15.349958
 15.374955]

heats
[0.071904538 0.067041971 0.07003089 0.0709173459999999

In [19]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[7.97022000e-04 1.25309829e-21 9.98746902e-19]
 [6.96715168e-04 1.42067857e-07 9.89808311e-05]
 [5.96788959e-04 3.31080932e-07 1.97585444e-04]
 [4.97269496e-04 5.94823724e-07 2.95787694e-04]
 [3.98210094e-04 9.88257831e-07 3.93534244e-04]
 [2.99734847e-04 1.63711688e-06 4.90700978e-04]
 [2.02203391e-04 2.90266270e-06 5.86928240e-04]
 [1.07179776e-04 6.35056336e-06 6.80651957e-04]
 [2.88986406e-05 2.62170626e-05 7.57637470e-04]
 [7.57118163e-06 1.02714952e-04 7.77673561e-04]
 [3.96708554e-06 1.96615504e-04 7.79990522e-04]
 [2.66668829e-06 2.92500628e-04 7.80007998e-04]
 [2.00508584e-06 3.88706984e-04 7.79390871e-04]
 [1.60565225e-06 4.84859501e-04 7.78515747e-04]
 [1.33862305e-06 5.80829957e-04 7.77512370e-04]
 [1.14761895e-06 6.76563508e-04 7.76437102e-04]]
(16, 3)


In [112]:
print(F_min)
print(dH_res_out)

0.0013997859794861785
[0.0 0.0 43.184070348285445]


In [61]:
# HOOK-JEEVES PROCEDURE

from copy import deepcopy

def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)

    #if F_min > new_min:
    F_min = new_min
    
    # exploratory_move
    for j in range(len(tar_num)):  
        print('input', lg_k[2], F_min)
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        #print('\na1 ', new_min, F_min)
        new_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                   _calor_setup_type_coeff)
        print('right', lg_k[2], new_min, F_min)
        #print('\na2 ', new_min, F_min)
        if F_min <= new_min:
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            new_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                  , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
            print('left', lg_k[2], new_min, F_min)
            #print('\na3 ', new_min, F_min)
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                #print('\na4 ', new_min, F_min)
            #else:
                
                #new_min = deepcopy(F_min)
                
        #else:
            
            #new_min = deepcopy(F_min)
    print('return', basis_point[2], lg_k[2])
    #print('\na5 ', new_min, F_min)
    return basis_point, lg_k, new_min

In [62]:
basis_point, lg_k, new_min = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff)
print(new_min)

for i in range(max_iter):
    print('\niteration', i, lg_k[2])
    if F_min <= new_min:
        
        if step_init < HJ_eps:
            print("p1 " + str(step_init) + " " + str(HJ_eps))
            break
            
        else: 
                 
            step_init /= 2
            
            basis_point, lg_k, new_min = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
            print('\nF min more ', new_min, F_min, lg_k[2])
            print(str(step_init))
    else:
        F_min = deepcopy(new_min)
        print('\nF min less', new_min, F_min, lg_k[2])
        print(str(step_init))
        basis_point_n = deepcopy(lg_k)
        lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

        new_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
        print('\nF min less after', new_min, F_min, lg_k[2])
        
        if F_min <= new_min:
            lg_k = deepcopy(basis_point_n)
            print('\nconstant less after', basis_point[tar_num], lg_k[2]) 
        else:
            F_min = new_min

        basis_point, lg_k, new_min = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
        
        if F_min <= new_min:
            
            print('\nF min p2', new_min, F_min, lg_k[2])
            if step_init < HJ_eps:
                print("p2")
                break
            
            else: 
                
                step_init /= 2
        
        else:

            F_min = deepcopy(new_min)
            
            print('\nF min p3.1', new_min, F_min, lg_k[2])
            
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                        con_matrix, ign_indices)
    
            new_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
            
            print('\nF min p3.2', new_min, F_min, lg_k[2])
            

[0.0 0.0 43.820814384706274]
input [7.] 0.0010193177691251117
[0.0 0.0 43.51750718192434]
right [7.5] 0.0011944344105393858 0.0010193177691251117
[0.0 0.0 44.31144142521809]
left [6.5] 0.0007647629763070088 0.0010193177691251117
return [7.] [6.5]
0.0007647629763070088

iteration 0 [6.5]

F min less 0.0007647629763070088 0.0007647629763070088 [6.5]
0.5
[0.0 0.0 45.07125046014062]

F min less after 0.00045272311719927907 0.0007647629763070088 [6.]
[0.0 0.0 45.07125046014062]
input [6.] 0.00045272311719927907
[0.0 0.0 44.31144142521809]
right [6.5] 0.0007647629763070088 0.00045272311719927907
[0.0 0.0 46.253608306548124]
left [5.5] 0.00016357966689789017 0.00045272311719927907
return [6.] [5.5]

F min p3.1 0.00016357966689789017 0.00016357966689789017 [5.5]
[0.0 0.0 48.17802186275707]

F min p3.2 6.204117962883576e-05 0.00016357966689789017 [5.]

iteration 1 [5.]

F min less 6.204117962883576e-05 6.204117962883576e-05 [5.]
0.5
[0.0 0.0 57.77842356593247]

F min less after 0.00147762498665

In [63]:
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

F_min, dH_res_out = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff)

print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

[0.0 0.0 47.759313786511825]

lg K
[[0.        ]
 [0.        ]
 [5.08898926]]

F_min
5.538680879232921e-05

Final enthalpies
[0.0 0.0 47.759313786511825]


In [34]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

lg_k is array

NameError: name 'array' is not defined